In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Load the dataset (using your provided data as an example)
df = pd.read_csv("F:\ML\Wine-Quality-Prediction\data\wine_quality_processed.csv")
X = df.drop(columns=['quality']).values
y = df['quality'].values

# Convert quality to categorical
encoder = OneHotEncoder(sparse_output=False)  # Updated parameter name
y_categorical = encoder.fit_transform(y.reshape(-1, 1))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Check shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")


X_train shape: (2318, 11), y_train shape: (2318, 6)


In [3]:
# Build the classification neural network
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(y_train.shape[1], activation='softmax')  # Output layer for classification
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=8, verbose=0, validation_split=0.1)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Test Loss: 0.5092, Test Accuracy: 0.8397


In [4]:
def activation_maximization_classification(target_class, model, iterations=500, learning_rate=0.01):
    # Start with a random input vector
    input_vector = tf.Variable(np.random.normal(size=(1, X_train.shape[1])), dtype=tf.float32)

    # Define the optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # Gradient ascent loop
    for i in range(iterations):
        with tf.GradientTape() as tape:
            # Predict probabilities for the input vector
            prediction = model(input_vector)
            # Loss is the negative log probability of the target class
            loss = -tf.math.log(prediction[0, target_class] + 1e-8)

        # Compute gradients and update input vector
        grads = tape.gradient(loss, input_vector)
        optimizer.apply_gradients([(grads, input_vector)])

        # Clip values to ensure realistic inputs
        input_vector.assign(tf.clip_by_value(input_vector, -3, 3))

    return input_vector.numpy().flatten()

In [ ]:
print("QUALITY 8 IS BEST QUALITY")
Best_Quality = 10
target_class = encoder.categories_[0].tolist().index(Best_Quality)  # Index of class '8'
optimized_input = activation_maximization_classification(target_class, model)
print(f"Optimized Input for Class {target_class} (Quality 8): {optimized_input}")

QUALITY 8 IS BEST QUALITY
Optimized Input for Class 5 (Quality 8): [ 0.9685086  -0.4744536   0.97511446  0.03763452 -0.24961634  0.78049004
  1.2581694  -1.070699   -2.5721092  -0.51179785 -0.1591086 ]


In [6]:
def counterfactual_explanation_classification(input_instance, target_class, model, iterations=500, learning_rate=0.01):
    # Convert the instance into a tensor
    input_tensor = tf.Variable(input_instance.reshape(1, -1), dtype=tf.float32)

    # Define the optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # Gradient descent loop
    for i in range(iterations):
        with tf.GradientTape() as tape:
            # Predict probabilities
            prediction = model(input_tensor)
            # Loss is the negative log probability of the target class + minimal changes
            loss = -tf.math.log(prediction[0, target_class] + 1e-8) + 0.01 * tf.reduce_sum(tf.square(input_tensor - input_instance))

        # Compute gradients and update input tensor
        grads = tape.gradient(loss, input_tensor)
        optimizer.apply_gradients([(grads, input_tensor)])

        # Clip the input values to realistic ranges
        input_tensor.assign(tf.clip_by_value(input_tensor, -3, 3))

    return input_tensor.numpy().flatten()


In [7]:
index = 0
instance = X_test[index]
true_class = np.argmax(y_test[index])
target_quality = 8 # Class '8' is the best quality
target_class = encoder.categories_[0].tolist().index(target_quality)  # Index of class '8'
modified_instance = counterfactual_explanation_classification(instance, target_class, model)
print(f"Original Instance: {instance}")
print(f"True Class: {true_class}")
print(f"Modified Instance for Class {target_class} (Quality {target_quality}): {modified_instance}")

Original Instance: [-0.29501569 -0.90237945  0.39774749 -0.5402045  -0.28850822 -0.09208832
 -0.72562543 -1.26820159 -0.49809235  0.2346359   1.12226075]
True Class: 5
Modified Instance for Class 5 (Quality 8): [-0.36350772 -0.8539945   0.42777157 -0.6565002  -0.26097444 -0.12002951
 -0.74048716 -1.2704254  -0.5103103   0.28759494  1.1331089 ]
